# Time Series modelling - y only - calculating rmse

In [36]:
import pandas as pd
import numpy as np

# ^^^ pyforest auto-imports - don't write above this line


from sklearn.model_selection import TimeSeriesSplit
from scipy import stats
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error


In [26]:
#Loading in combined train and test set

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df["train_test"] = 'train'
df_test["train_test"] = "test"

df_combined = pd.concat([df,df_test])

/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [27]:
#Linear interpolation on all features

def interpolator(df):
    for g in df.galaxy.unique():
        #splitting out the individual galaxies
        df_temp = df[df['galaxy']==g].copy()
        df = df[~(df['galaxy']==g)]
        #filling the Nans (first with interpolation and then backfilling if missing for start)
        df_temp.sort_values("galactic year",inplace=True)
        df_temp.interpolate(method='linear',inplace=True)
        df_temp.fillna(method ='bfill',inplace=True)
        df_temp.fillna(method ='ffill',inplace=True)
        #adding back to dataframe
        df = pd.concat([df,df_temp])
    #filling the Nans for any which didnt have anything to fill at all
    df.fillna(df.mean(),inplace=True)
    return df

df_combined = interpolator(df_combined)
#df_combined = pd.get_dummies(df_combined, columns=['galaxy'])
df = df_combined[df_combined["train_test"]=='train'].copy()
df_test = df_combined[df_combined["train_test"]=='test'].copy()
df.drop('train_test',axis=1,inplace=True)
df_test.drop('train_test',axis=1,inplace=True)


In [28]:
#Fitting galactic year to date time format

df['galactic year 1'] = df['galactic year'].apply(lambda x:(np.round((x/1000),0))*2)
df['galactic year 1'] = df['galactic year 1'].astype('int64')
df['galactic year 1'] = df['galactic year 1'].astype('str')
df['galactic year 1'] = df['galactic year 1'] + "0101"
df['date'] = pd.to_datetime(df['galactic year 1'], yearfirst=True
                                 , format='%Y/%m/%d'
                                )
df.head()

,Adjusted net savings,"Adolescent birth rate (births per 1,000 female creatures ages 15-19)","Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Current health expenditure (% of GGP),Domestic credit provided by financial sector (% of GGP),Education Index,Employment in agriculture (% of total employment),Employment in services (% of total employment),Employment to population ratio (% ages 15 and older),"Estimated gross galactic income per capita, female",...,Vulnerable employment (% of total employment),Young age (0-14) dependency ratio (per 100 creatures ages 15-64),Youth unemployment rate (female to male ratio),existence expectancy at birth,existence expectancy index,galactic year,galaxy,y,galactic year 1,date
0,43.391708,174.617899,6.313144,6.248891,75.604799,0.526375,96.827657,53.392606,76.506783,16498.415777,...,91.911151,98.216072,2.184798,63.125200,0.628657,990025,Large Magellanic Cloud (LMC),0.052590,19800101,1980-01-01
194,43.391708,165.945907,6.313144,6.248891,73.837393,0.526375,96.827657,53.392606,76.506783,16498.415777,...,91.911151,95.964868,2.184798,62.389059,0.627245,991020,Large Magellanic Cloud (LMC),0.052115,19820101,1982-01-01
410,43.391708,157.273915,6.313144,6.248891,72.069987,0.526375,95.022335,50.937661,76.302787,16498.415777,...,93.217778,93.713664,2.114685,68.794245,0.662135,992016,Large Magellanic Cloud (LMC),0.052006,19840101,1984-01-01
543,43.391708,148.601923,6.313144,6.248891,70.302580,0.526375,93.217014,48.482717,76.098791,16498.415777,...,94.524405,91.462460,2.044571,62.848660,0.687776,993012,Large Magellanic Cloud (LMC),0.051675,19860101,1986-01-01
765,43.391708,139.929931,6.313144,6.248891,68.535174,0.526375,91.411692,46.027772,75.894795,16498.415777,...,95.831032,89.211257,1.974458,67.482431,0.727717,994009,Large Magellanic Cloud (LMC),0.051334,19880101,1988-01-01


In [54]:
#Testing on one galaxy

split_1_y = []
split_2_y = []
split_0_y = []
split_1_preds = []
split_2_preds = []
split_0_preds = []


for g in df.galaxy.unique()[:1]:
    
    print(g)
    df_temp = df[df['galaxy']==g].copy()
    
    df_temp.sort_values("galactic year 1",inplace=True)
#             df_temp['galactic year'] = df_temp['galactic year'].apply(lambda x:(np.round((x/1000),0))*2)
#             df_temp['galactic year'] = df_temp['galactic year'].astype('int64')
#             df_temp['galactic year'] = df_temp['galactic year'].astype('str')
#             df_temp['galactic year'] = df_temp['galactic year'] + "0101"
#             df_temp['date'] = pd.to_datetime(df_temp['galactic year'], yearfirst=True
#                                              , format='%Y/%m/%d'
#                                             )

    df_temp_temp = df_temp[['date','y','galaxy','galactic year 1']]
    df_temp_temp_1 = df_temp_temp[['date','y']].copy()
    df_temp_temp_1.set_index('date', inplace=True, drop=True)
    data = df_temp_temp_1
    

    ts = TimeSeriesSplit(n_splits=3)
    splits = [split for split in ts.split(data)]
    test_size = splits[0][1].shape[0]

    model = sm.tsa.statespace.SARIMAX(data, order=(1, 0, 0))

    y_train = data.iloc[splits[0][0]]
    y_test = data.iloc[splits[0][1]]
    model_cv = sm.tsa.statespace.SARIMAX(y_train, order=(1, 0, 0))
    results_cv = model_cv.fit(maxiter=5000)
    pred_test = results_cv.predict(start=0, end=len(y_train)+test_size-1, dynamic=False)[-test_size:]
    print(mean_squared_error(y_test, pred_test))
    split_0_y.append(y_test.y)
    split_0_preds.append(pred_test)


    y_train = data.iloc[splits[1][0]]
    y_test = data.iloc[splits[1][1]]
    model_cv = sm.tsa.statespace.SARIMAX(y_train, order=(1, 0, 0))
    results_cv = model_cv.fit(maxiter=5000)
    pred_test = results_cv.predict(start=0, end=len(y_train)+test_size-1, dynamic=False)[-test_size:]
    split_1_y.append(y_test.y)
    split_1_preds.append(pred_test)



    y_train = data.iloc[splits[2][0]]
    y_test = data.iloc[splits[2][1]]
    model_cv = sm.tsa.statespace.SARIMAX(y_train, order=(1, 0, 0))
    results_cv = model_cv.fit(maxiter=5000)
    pred_test = results_cv.predict(start=0, end=len(y_train)+test_size-1, dynamic=False)[-test_size:]
    split_2_y.append(y_test.y)
    split_2_preds.append(pred_test)

Large Magellanic Cloud (LMC)
2.250789737638028e-09


/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/st

In [55]:
len(split_0_y)

1

In [56]:
split_0_preds

[1994-01-01    0.050130
 1996-01-01    0.049789
 1998-01-01    0.049450
 2000-01-01    0.049113
 Freq: 2AS-JAN, dtype: float64]

In [61]:
mses = []

mses.append(mean_squared_error(split_0_y, split_0_preds))
mses.append(mean_squared_error(split_1_y, split_1_preds))
mses.append(mean_squared_error(split_2_y, split_2_preds))

mses
print(np.mean(mses))

3.604368608365351e-05


In [74]:
#Fitting time series on each galaxy


#Train and test set

#df_combined['preds'] = df_combined['y'].apply(lambda x:0)

pd.reset_option('display.max_rows')




results_df = pd.DataFrame(columns=['date','y','preds','galaxy','galactic year 1'])
error_list = []
split_1_y = []
split_2_y = []
split_0_y = []
split_1_preds = []
split_2_preds = []
split_0_preds = []

for g in df.galaxy.unique():
    try:
        print(g)
        df_temp = df[df['galaxy']==g].copy()
        df_temp.sort_values("galactic year 1",inplace=True)
#             df_temp['galactic year'] = df_temp['galactic year'].apply(lambda x:(np.round((x/1000),0))*2)
#             df_temp['galactic year'] = df_temp['galactic year'].astype('int64')
#             df_temp['galactic year'] = df_temp['galactic year'].astype('str')
#             df_temp['galactic year'] = df_temp['galactic year'] + "0101"
#             df_temp['date'] = pd.to_datetime(df_temp['galactic year'], yearfirst=True
#                                              , format='%Y/%m/%d'
#                                             )

        df_temp_temp = df_temp[['date','y','galaxy','galactic year 1']]
        df_temp_temp_1 = df_temp_temp[['date','y']].copy()
        df_temp_temp_1.set_index('date', inplace=True, drop=True)
        data = df_temp_temp_1

        ts = TimeSeriesSplit(n_splits=3)
        splits = [split for split in ts.split(data)]
        test_size = splits[0][1].shape[0]

        model = sm.tsa.statespace.SARIMAX(data, order=(1, 0, 0))

        y_train = data.iloc[splits[0][0]]
        y_test = data.iloc[splits[0][1]]
        model_cv = sm.tsa.statespace.SARIMAX(y_train, order=(1, 0, 0))
        results_cv = model_cv.fit(maxiter=5000)
        pred_test = results_cv.predict(start=0, end=len(y_train)+test_size-1, dynamic=False)[-test_size:]
        print(mean_squared_error(y_test, pred_test))
        for x in y_test.y:
            split_0_y.append(x)
        for x in pred_test:
            split_0_preds.append(x)


        y_train = data.iloc[splits[1][0]]
        y_test = data.iloc[splits[1][1]]
        model_cv = sm.tsa.statespace.SARIMAX(y_train, order=(1, 0, 0))
        results_cv = model_cv.fit(maxiter=5000)
        pred_test = results_cv.predict(start=0, end=len(y_train)+test_size-1, dynamic=False)[-test_size:]
        for z in y_test.y:
            split_1_y.append(z)
        for z in pred_test:
            split_1_preds.append(z)



        y_train = data.iloc[splits[2][0]]
        y_test = data.iloc[splits[2][1]]
        model_cv = sm.tsa.statespace.SARIMAX(y_train, order=(1, 0, 0))
        results_cv = model_cv.fit(maxiter=5000)
        pred_test = results_cv.predict(start=0, end=len(y_train)+test_size-1, dynamic=False)[-test_size:]
        for a in y_test.y:
            split_2_y.append(a)
        for a in pred_test:
            split_2_preds.append(a)


    except:
        None
        print('ERROR:',g)
        error_list.append(g)
#             df_temp = df[df['galaxy']==g].copy()
#             df = df[~(df['galaxy']==g)]
#             df_temp.sort_values("galactic year",inplace=True)
#             df_temp['preds'] = df_temp['y']
#             df = pd.concat([df,df_temp])
print(error_list)

    
    


Large Magellanic Cloud (LMC)
2.250789737638028e-09
Camelopardalis B


/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/st

2.76370201301066e-07
Virgo I
7.251181850358619e-06


/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/st

UGC 8651 (DDO 181)
1.2144073004488743e-06
Tucana Dwarf


...

/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:949: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/Tiffany/

NGC 5206
1.2219910965329034e-06
NGC 5102


/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsm

3.485837925174698e-06
['NGC 5253']


/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:949: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/Tiffany/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency 2AS-JAN will be used.
  % freq, ValueWarning)


In [70]:
split_1_y

[                   y
 date                
 2002-01-01  0.048314
 2004-01-01  0.047988
 2006-01-01  0.047607
 2008-01-01  0.047392,                    y
 date                
 2006-01-01  0.057700
 2008-01-01  0.057908
 2010-01-01  0.057663
 2012-01-01  0.056803
 2014-01-01  0.056656,                    y
 date                
 2004-01-01  0.042873
 2006-01-01  0.042461
 2008-01-01  0.042236
 2010-01-01  0.041875
 2012-01-01  0.040427
 2014-01-01  0.040060,                    y
 date                
 2004-01-01  0.042353
 2006-01-01  0.041920
 2008-01-01  0.041655
 2010-01-01  0.041258
 2012-01-01  0.039067
 2014-01-01  0.038659,                    y
 date                
 2008-01-01  0.176438
 2010-01-01  0.177138
 2012-01-01  0.186606
 2014-01-01  0.187557
 2016-01-01  0.187805
 2018-01-01  0.188126,                    y
 date                
 2002-01-01  0.045243
 2004-01-01  0.044807
 2006-01-01  0.044381
 2008-01-01  0.044107
 2010-01-01  0.043713,                    y
 date     

In [75]:
mses = []

mses.append(mean_squared_error(split_0_y, split_0_preds))
mses.append(mean_squared_error(split_1_y, split_1_preds))
mses.append(mean_squared_error(split_2_y, split_2_preds))

mses
print(np.mean(mses))

0.0003476252901659671


In [76]:
len(split_2_y)
len(split_2_preds)

897

In [78]:
df[df['galaxy']=='NGC 5253']

,Adjusted net savings,"Adolescent birth rate (births per 1,000 female creatures ages 15-19)","Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Current health expenditure (% of GGP),Domestic credit provided by financial sector (% of GGP),Education Index,Employment in agriculture (% of total employment),Employment in services (% of total employment),Employment to population ratio (% ages 15 and older),"Estimated gross galactic income per capita, female",...,Vulnerable employment (% of total employment),Young age (0-14) dependency ratio (per 100 creatures ages 15-64),Youth unemployment rate (female to male ratio),existence expectancy at birth,existence expectancy index,galactic year,galaxy,y,galactic year 1,date
144,20.751377,114.074664,6.313144,8.536079,119.963809,0.750846,55.203046,67.082939,69.150998,22013.091432,...,69.460892,79.295416,2.257781,77.382893,0.882087,990025,NGC 5253,0.045115,19800101,1980-01-01
